In [16]:
import arcpy
import os
from os.path import exists
import shutil
import numpy as np
from arcpy.sa import *
from arcpy import env 
##ArcGIS
aprx = arcpy.mp.ArcGISProject("CURRENT")
aprx_map = aprx.listMaps("Map")[0]
arcpy.env.overwriteOutput = True
coordinate_system = 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'


##Paths
gdb_path = r"D:\NIWC_Eco\ArcGIS_Project\NIWC_Eco.gdb"
deliverables_path = r"D:\NIWC_Eco\DELIVERABLES"
eco_tables_directory_path = r"D:\NIWC_Eco\EcoData\Eco_Tables"
atr_tables_directory_path = r"D:\NIWC_Eco\ATR\ATR_Tables"
atr_reports_directory_path = os.path.join(deliverables_path, 'ATR_REPORTS')


##Directories
eco_table_dir = os.listdir(eco_tables_directory_path)
atr_dir = os.listdir(atr_tables_directory_path)
##Lists
ctd_list = arcpy.ListFeatureClasses("*_C", "Point")
kearfott_list = arcpy.ListFeatureClasses("*_K", 'Point')
flntu_list = arcpy.ListFeatureClasses("*_F", 'Point')

ctd_value_list = ['temperature', 'salinity', "sound_speed", 'depth']
flntu_value_list = ['turbidity', 'chlorophyll']
kearfott_value_list = ['altitude']

p_list = arcpy.ListFeatureClasses("P*", "Point")
idw_list = aprx_map.listLayers('I_*')
extent_list = arcpy.ListFeatureClasses("E*", 'Polygon')
raster_list = arcpy.ListRasters()
raster_list_map = aprx_map.listLayers('R_*')
table_list = arcpy.ListTables()
layer_list = aprx_map.listLayers()
atr_p_list = arcpy.ListFeatureClasses('A_*', 'Point')
atr_p_list_map = aprx_map.listLayers('A_*')

points_list = arcpy.ListFeatureClasses()

##IDW Environment
cell_size = 5.5E-06
power = 1
search_neighborhood =  "NBRTYPE=Smooth S_MAJOR=4.50389212633055E-03 S_MINOR=4.50389212633055E-03 ANGLE=0 SMOOTH_FACTOR=0.2"

##Break Values
breakValTemp = [12.0, 13.0, 14.0, 15.0, 20.0]
breakValSal= [32, 32.5, 33, 33.5, 35]
breakValDep = [10.0, 11.8, 13.5, 15.2, 17.0]
breakValSS = [1497.0, 1501, 1506, 1510, 1518.0]
breakValTurb = [0.5, 1, 1.5, 2, 3]
breakValCHL = [1.5, 3, 4.5, 6, 10]
#breakValCHL = [0.4, 0.5, 0.6, 0.7, 0.8]
breakValAlt = [4,4.5,5, 5.5, 6]
breakLab = ['Very Low', 'Low', 'Normal', 'High', 'Very High']

colormap = [[0, 61, 161, 209], [1, 170, 206, 172], [2, 241, 251, 123], [3, 247, 162, 72], [4, 240, 38, 28]]

            
'''def add_tables():
    print("\n Adding Tables \n")
    arcpy.env.workspace = gdb_path
    print(table_list)
    for i in eco_table_dir:
        i_split = i.split(".")[0]
        if i in table_list:
            print(f"!!{i} has already been added to map!!")
        elif i_split in table_list:
            print(f"!!{i_split} has already been added to map!!")
        else: 
            arcpy.env.workspace = eco_tables_directory_path
            print(f"**Adding {i}")
            arcpy.conversion.TableToGeodatabase(i, gdb_path)'''

            


def generate_points():
    print("\n Creating Points \n")
    for i in eco_table_dir:
        i_path = os.path.join(eco_tables_directory_path, i)
        i_name = i.split(".")[0]
        value = i.split("_")[2]
        
        arcpy.env.workspace = gdb_path
        points_list = arcpy.ListFeatureClasses("*", "Points") 

        if value == 'Kearfott':
            out_i = 'P_' + i[0:16] + '_K'
            if out_i not in points_list:
                print(f"**Creating {i_name} points")
                arcpy.defense.CoordinateTableToPoint(i_path, out_i, 'longitude', "DD_2", 'latitude', coordinate_system)

        if value == 'CTD':
            out_i = 'P_' + i[0:16] + '_C'
            if out_i not in points_list:
                print(f"**Creating {i_name} points")
                arcpy.defense.CoordinateTableToPoint(i_path, out_i, 'longitude', "DD_2", 'latitude', coordinate_system)
                
        if value == 'FLNTU':
            out_i = 'P_' + i[0:16] + '_F'
            if out_i not in points_list:
                print(f"**Creating {i_name} points")
                arcpy.defense.CoordinateTableToPoint(i_path, out_i, 'longitude', "DD_2", 'latitude', coordinate_system)
                

def create_atr_points():
    print("\n Creating ATR Points \n")
    arcpy.env.workspace = gdb_path
    for i in atr_dir:
        i_path = os.path.join(atr_tables_directory_path, i)
        value = i.split("_")[0]
        if value == 'ATR':
            i_clean = i.split('_')[1]
            i_clean = i_clean.split('.')[0]
            out_i = 'A_' + (i_clean)
            if out_i in points_list:
                print(f"!!{out_i} Points Already Exists!!")
            else:
                print(f"**Creating {out_i} points")
                arcpy.defense.CoordinateTableToPoint(i_path, out_i, 'longitude', "DD_2", 'latitude', coordinate_system)
                
def alter_field():
    arcpy.env.workspace = gdb_path
    p_list = arcpy.ListFeatureClasses("P*", "Point")
    print("\n Altering Fields \n")
    for p in p_list:
        value_abv = p.split('_')[-1]
        if value_abv == 'K': 
            fields_list = arcpy.ListFields(p, '*alt', 'Double')
            for f in fields_list:
                #print(f.name)
                if f.name == 'kearfott_alt':
                    print('Altering Altitude Field Name')
                    arcpy.management.AlterField(p, "kearfott_alt", "altitude", 'altitude', "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

        if value_abv == 'F':
            fields_list = arcpy.ListFields(p, 'ch*', "Double")
            for f in fields_list:
                if f.name == 'chlorophyll_a':
                    print('Altering Chlorophyll Field Name')
                    arcpy.management.AlterField(p, "chlorophyll_a", "chlorophyll", 'chlorophyll', "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR") 
            
        
        
def shutil_tables():
    arcpy.env.workspace = eco_tables_directory_path
    print("\n Moving Tables to Processed Directory" )
    for i in eco_table_dir:
        src = os.path.join(eco_tables_directory_path, i)
        processed_path = r"D:\NIWC_Eco\EcoData\Processed_Tables"
        dst = os.path.join(processed_path, i)
        try:
            shutil.move(src, dst)
        except Exception as e:
            print(f"{i} is already in Processed Directory")

            
            
def shutil_atr():
    arcpy.env.workspace = atr_tables_directory_path
    print("\n Moving ATR Tables to Processed ATR Directory \n")
    processed_atr_path = r"D:\NIWC_Eco\ATR\Processed_ATR"
    for i in atr_dir:
        src = os.path.join(atr_tables_directory_path, i)
        dst = os.path.join(processed_atr_path, i)
        shutil.move(src, dst)
                
            
    
def trim_points():
    arcpy.env.workspace = gdb_path
    print("\n Trimming Points \n")
    for i in p_list:
        value = i.split("_")[3]
        if value == 'K':
            print(f'Updating {i} Kearfott Tables')
            field = ["altitude"]
            with arcpy.da.UpdateCursor(i, field) as cursor:
                for row in cursor:
                    if(row[0] < 0 or row[0] > 7):
                        cursor.deleteRow()

        elif value == 'C':
            fields = ["temperature", "salinity", "sound_speed", 'depth']
            print(f'Updating {i} CTD Tables')
            with arcpy.da.UpdateCursor(i, fields) as cursor:
                for row in cursor:
                    if(row[0] < 0 or row[0] > 30):
                        cursor.deleteRow()
                    if(row[1] < 30 or row[1] > 40):
                        cursor.deleteRow()
                    if(row[2] < 0):
                        cursor.deleteRow()
                    if(row[3] < 12):
                        cursor.deleteRow()
        elif value == 'F':
            field = ['depth']
            print(f'Updating {i} FLNTU Tables')
            with arcpy.da.UpdateCursor(i, field) as cursor:
                for row in cursor:
                    if(row[0] < 12 or row[0] > 25):
                        cursor.deleteRow()

                    
def extent():
    arcpy.env.workspace = gdb_path
    print("\n Creating Extents \n")
    for i in p_list:
        mission_name = i[2:18]
        out_i = 'E_' + mission_name
        if out_i not in extent_list:
            print(f"**Creating {out_i} Extent")
            arcpy.management.MinimumBoundingGeometry(i, out_i, "CONVEX_HULL", "ALL", "", "MBG_FIELDS")

            
            
            
            
def IDW():
    arcpy.env.workspace = gdb_path
    print('\n Creating IDWs \n')
    for i in p_list:
        mission_name = i[2:18]
        value = i.split("_")[3] 
        if value == 'K':
            for e in extent_list:
                check_name = 'E_' + mission_name
                if e == check_name:
                    for j in kearfott_value_list:
                        j_cap = j.capitalize()
                        out_i = 'I_' + mission_name + '_' + j_cap
                        out_ras = 'R_' + mission_name + '_' + j_cap
                        out_con = 'C_' + mission_name + '_' + j_cap
                        if out_ras in raster_list:
                            print(f"!!{out_i} IDW and Raster have already been created!!")
                        else:
                            print(f"**Creating {out_i} IDW ")
                            with arcpy.EnvManager(outputCoordinateSystem= coordinate_system, mask=e):
                                arcpy.ga.IDW(i, j, out_i, out_ras,cell_size,power,search_neighborhood,j)
                            
        if value == 'C':
            for e in extent_list:
                check_name = 'E_' + mission_name
                if e == check_name:
                    for j in ctd_value_list:
                        j_cap = j.capitalize()
                        out_i = 'I_' + mission_name + '_' + j_cap
                        out_ras = 'R_' + mission_name + '_' + j_cap
                        out_con = 'C_' + mission_name + '_' + j_cap
                        if out_ras in raster_list:
                            print(f"!!{out_i} IDW and Raster have already been created!!")
                        else:
                            print(f"**Creating {out_i} IDW ")
                            with arcpy.EnvManager(outputCoordinateSystem= coordinate_system, mask=e):
                                arcpy.ga.IDW(i, j, out_i, out_ras,cell_size,power,search_neighborhood,j)

                            
        if value == 'F':
            for e in extent_list:
                check_name = 'E_' + mission_name
                if e == check_name:
                    for j in flntu_value_list:
                        j_cap = j.capitalize()
                        out_i = 'I_' + mission_name + '_' + j_cap
                        out_ras = 'R_' + mission_name + '_' + j_cap
                        
                        if out_ras in raster_list:
                            print(f"!!{out_i} IDW and Raster have already been created!!")
                        else:
                            print(f"**Creating {out_i} IDW ")
                            with arcpy.EnvManager(outputCoordinateSystem= coordinate_system, mask=e):
                                arcpy.ga.IDW(i, j, out_i, out_ras,cell_size,power,search_neighborhood,j)
                                
                                




def make_directories():
    arcpy.env.workspace = gdb_path 
    print("\n Validating & Creating TIFF Directories \n")
    file_type_list = ['TIFF', 'SHP', 'ATR']
    raster_list = arcpy.ListRasters()
    for raster in raster_list:
        for file in file_type_list:
            #print(raster)
            no_prefix = raster[2:]
            mission_name = raster[2:18]
            variable = raster[19:]
            mission_path = os.path.join(deliverables_path, mission_name) 
            data_path = os.path.join(mission_path, file)
            variable_path = os.path.join(data_path, variable)

            mpe = os.path.exists(mission_path)
            dpe = os.path.exists(data_path)
            vpe = os.path.exists(variable_path)
            
            if mpe == False:
                print(f"Creating {mission_path}")
                os.mkdir(mission_path)

            if dpe == False:
                print(f"Creating {data_path}")
                os.mkdir(data_path)

            if vpe == False:
                if file != 'ATR':
                    print(f"Creating {variable_path}")
                    os.mkdir(variable_path)






def export_contours():
    shp_directory = os.path.join(mission_directory, 'SHP')
    shp_value_dir = os.path.join(shp_directory, valueCap)
    for idw in idw_list:
        mission_name = idw[2:18]
        idw_split = idw[19:]
        print(f"Creating {mission_name} {value} Contour ")
        with arcpy.EnvManager(outputCoordinateSystem= coordinate_system):
            if idw_split == "Chlorophyll":
                classes_breaks = breakValCHL
            if idw_split == 'Turbidity':
                classes_breaks = breakValTurb
            if idw_split == 'Temperature':
                classes_breaks = breakValTemp
            if idw_split == 'Altitude':
                classes_breaks = breakValAlt
            if idw_split == 'Sound_speed':
                classes_breaks = breakValSS
            if idw_split == 'Depth':
                classes_breaks = breakValDep
            if idw_split == 'Salinity':
                classes_breaks = breakValSal
            
            arcpy.ga.GALayerToContour(in_geostat_layer=idw,
                                      contour_type = 'SAME_AS_LAYER', 
                                      out_feature_class = out_con,
                                      contour_quality = 'PRESENTATION',
                                      classification_type = 'MANUAL',
                                      classes_breaks = classes_breaks)

            


def calculate_field():
    arcpy.env.workspace = gdb_path
    print("\n Creating & Calculating 'Name' Field \n")
    atr_p_list = arcpy.ListFeatureClasses('A_*', 'Point')
    for i in atr_p_list:
        i_split = i.split('_')[1]
        fields_list = arcpy.ListFields(i)
        for f in fields_list:
            field = arcpy.Describe(f)
            field = field.name
        if field not in fields_list:
            print("**Creating 'Name' Field")
            arcpy.management.AddField(i, 'Name', 'TEXT')
            arcpy.management.CalculateField(i, 'Name', '"'+i_split+'"', 'PYTHON3')
            arcpy.management.CalculateField(i, 'Mission', "!Mission!.replace('-', '_')")
            
            
            
            
                
def split_atr():
    arcpy.env.workspace = gdb_path
    print("\n Spliting ATR Points \n")
    arcpy.env.overwriteOutput = False
    atr_p_list = arcpy.ListFeatureClasses('A_*', 'Point')
    for i in atr_p_list:
        value = i.split('_')[1]
        try:
            print(f"**Splitting {i}")
            arcpy.analysis.SplitByAttributes(i, gdb_path, ['Mission', 'Name'])
        except Exception as e:
            print(f"!!{i} has already been split!!")


                

def extract_values():
    arcpy.env.workspace = gdb_path
    print("\n Extracting Multi-Values to ATR Calls \n")
    split_list = arcpy.ListFeatureClasses('SN*', 'Point')
    print(split_list)
    for i in split_list:
        #print(i)
        i_suffix = i.split("_")[2]
        mission_name = i[0:16]
        #print(i_suffix)
        #print(mission_name)
        for j in raster_list:
            validation = j[2:18]
            value = j[19:]
            rasterList = j +" "+ value
            if validation == mission_name:
                #print(f'{i} {value}')
                ExtractMultiValuesToPoints(i, rasterList)
                



                            

def table_excel():
    arcpy.env.workspace = gdb_path
    print("\n Exporting ATR Calls Report \n")
    #print(split_list)
    split_list = arcpy.ListFeatureClasses('SN*', 'Point')
    for i in split_list:
        print(i)
        mission_name = i[0:16]
        suffix_split = i.split('_')[2]
        out_name = i
        field_list = arcpy.ListFields(i)
        atr_reports_directory_path = "{}\{}\ATR\{}.xlsx".format(deliverables_path, mission_name, i)
        arpe = os.path.exists(atr_reports_directory_path)
        try:
            if suffix_split == 'DEEPMINE' and arpe == False:
                deepmine_directory_path = os.path.join(atr_reports_directory_path, "DEEPMINE")
                Output_Excel_File = os.path.join(deepmine_directory_path, out_name)+'.xlsx'
                print(f"Moving {i} ----> {atr_report_directory_path}")
                arcpy.conversion.TableToExcel(i, atr_reports_directory_path)
                print(f"Deleting {i}")
                arcpy.management.Delete(i)
        except Exception as e:
            arcpy.management.Delete(i)
            
        try:
            if suffix_split == 'SEEBYTE' and arpe == False:
                seebyte_directory_path = os.path.join(atr_reports_directory_path, "SEEBYTE")
                Output_Excel_File = os.path.join(seebyte_directory_path, out_name)+'.xlsx'
                print(f"Moving {i} ----> {atr_report_directory_path}")
                arcpy.conversion.TableToExcel(i, atr_reports_directory_path)
                print(f"Deleting {i}")
                arcpy.management.Delete(i)
        except Exception as e:
            arcpy.management.Delete(i)
            
        try:   
            if suffix_split == 'NATR' and arpe == False:
                natr_directory_path = os.path.join(atr_reports_directory_path, "NATR")
                Output_Excel_File = os.path.join(natr_directory_path, out_name)+'.xlsx'
                print(f"Moving {i} ----> {atr_report_directory_path}")
                arcpy.conversion.TableToExcel(i, atr_reports_directory_path)
                print(f"Deleting {i}")
                arcpy.management.Delete(i)
            
        except Exception as e:
            arcpy.management.Delete(i)
        
        try:
            if suffix_split == 'XATR'  and arpe == False: 
                xatr_directory_path = os.path.join(atr_reports_directory_path, "XATR")
                Output_Excel_File = os.path.join(xatr_directory_path, out_name)+'.xlsx'
                print(f"Moving {i} ----> {atr_report_directory_path}")
                arcpy.conversion.TableToExcel(i, atr_reports_directory_path)
                print(f"Deleting {i}")
                arcpy.management.Delete(i)

        except Exception as e:
            arcpy.management.Delete(i)

def delete_atr():
    arcpy.env.workspace = gdb_path
    print("\n Deleting Unsplit ATR Points")
    atr_p_list = arcpy.ListFeatureClasses('A_*', 'Point')
    for i in atr_p_list:
        arcpy.management.Delete(i)
        
def delete_split_atr():
    arcpy.env.workspace = gdb_path
    print("\n Deleting Split ATR Points")
    for i in split_list:
        arcpy.management.Delete(i)        
            
    
    
    

'''group_list = ['FLNTU', 'CTD', 'Kearfott', 'FLNTU Extent', 'CTD Extent', 'Kearfott Extent', 'Contour', 'Raster', 'IDW']
sub_group_list = ['Chlorophyll', 'Turbididty', 'Altitude', 'Depth', 'Sound Speed', 'Salinity', 'Temperature']
    
def group_layers():
    print("\n Grouping Points \n")
    #print(kearfott_group)
    for l in layer_list:
        l_prefix = l[0]
        if l_prefix == 'R':
            aprx_map.addLayerToGroup()
        for p in kearfott_points:
            #print(lyr)
            print(f'Adding {lyr} to Kearfott Group')
            aprx_map.addLayerToGroup(lyr, pt)
            aprx_map.removeLayer(pt)       
        '''


    
                
if __name__ == "__main__":
    generate_points()
    create_atr_points()
    #delete_split_atr()
    alter_field()
    trim_points()
    extent()
    IDW()
    make_directories()
    #delete_points()
    #raster_symbology()
    #export_raster()
    #export_contours()
    calculate_field()
    split_atr()
    extract_values()
    table_excel()
    #delete_extent()
    #delete_IDW()
    #delete_points()
    #delete_atr()
    shutil_atr()
    shutil_tables()
    delete_atr()
    #delete_split_atr()
    
   


 Creating Points 


 Creating ATR Points 


 Altering Fields 


 Trimming Points 

Updating P_SN06255_20150211_C CTD Tables
Updating P_SN06255_20150211_F FLNTU Tables
Updating P_SN06255_20150211_K Kearfott Tables
Updating P_SN06255_20150212_C CTD Tables
Updating P_SN06255_20150212_F FLNTU Tables
Updating P_SN06255_20150212_K Kearfott Tables
Updating P_SN06255_20150304_C CTD Tables
Updating P_SN06255_20150304_F FLNTU Tables
Updating P_SN06255_20150304_K Kearfott Tables
Updating P_SN06255_20150306_C CTD Tables
Updating P_SN06255_20150306_F FLNTU Tables
Updating P_SN06255_20150306_K Kearfott Tables
Updating P_SN06255_20150309_C CTD Tables
Updating P_SN06255_20150309_F FLNTU Tables
Updating P_SN06255_20150309_K Kearfott Tables
Updating P_SN06255_20150402_C CTD Tables
Updating P_SN06255_20150402_F FLNTU Tables
Updating P_SN06255_20150402_K Kearfott Tables
Updating P_SN06255_20150406_C CTD Tables
Updating P_SN06255_20150406_F FLNTU Tables
Updating P_SN06255_20150406_K Kearfott Tables
Updati

SN06255_20150612_DEEPMINE Chlorophyll
SN06255_20150612_DEEPMINE Altitude
SN06255_20150615_DEEPMINE Temperature
SN06255_20150615_DEEPMINE Salinity
SN06255_20150615_DEEPMINE Sound_speed
SN06255_20150615_DEEPMINE Depth
SN06255_20150615_DEEPMINE Turbidity
SN06255_20150615_DEEPMINE Chlorophyll
SN06255_20150615_DEEPMINE Altitude
SN06255_20150720_DEEPMINE Temperature
SN06255_20150720_DEEPMINE Salinity
SN06255_20150720_DEEPMINE Sound_speed
SN06255_20150720_DEEPMINE Depth
SN06255_20150720_DEEPMINE Turbidity
SN06255_20150720_DEEPMINE Chlorophyll
SN06255_20150720_DEEPMINE Altitude
SN06255_20150723_DEEPMINE Temperature
SN06255_20150723_DEEPMINE Salinity
SN06255_20150723_DEEPMINE Sound_speed
SN06255_20150723_DEEPMINE Depth
SN06255_20150723_DEEPMINE Turbidity
SN06255_20150723_DEEPMINE Chlorophyll
SN06255_20150723_DEEPMINE Altitude
SN06255_20150724_DEEPMINE Temperature
SN06255_20150724_DEEPMINE Salinity
SN06255_20150724_DEEPMINE Sound_speed
SN06255_20150724_DEEPMINE Depth
SN06255_20150724_DEEPMINE Tu

In [2]:
print('hello world')

'''def raster_symbology():
    print("\n Altering Raster Symbology \n")
    arcpy.env.workspace = aprx_map                                            
    for l in raster_list_map:
        print(l)
        desc = arcpy.Describe(l)
        descName = desc.nameString
        descSplit = descName[19:]
        sym = l.symbology
        if hasattr(sym, 'colorizer'):  
            try:
                if sym.colorizer.type == 'RasterClassifyColorizer':
                    sym.colorizer.breakCount = 5
                    sym.colorizer.classificationMethod = 'ManualInterval'
                    count= 0
                    #print(f"Fixing {l}")
                    #sym.colorizer.classBreaks = classBreaks_Temperature
                    if descSplit == "Temperature":
                        print(f'Adjusting {descSplit}')
                        for brk in sym.colorizer.classBreaks:
                            brk.upperBound = breakValTemp[count]
                            brk.label = breakLab[count]
                            count += 1
                            #print(f"Temp{count}, {count}")

                    if descSplit == 'Salinity':
                        print(f'Adjusting {descSplit}')
                        for brk in sym.colorizer.classBreaks:
                            brk.upperBound = breakValSal[count]
                            brk.label = breakLab[count]
                            count += 1

                    if descSplit == 'Depth':
                        print(f'Adjusting {descSplit}')
                        for brk in sym.colorizer.classBreaks:
                            brk.upperBound = breakValDep[count]
                            #brk.label = breakLab[count]
                            count+= 1

                    if descSplit == 'Sound_speed':
                        print(f'Adjusting {descSplit}')
                        for brk in sym.colorizer.classBreaks:
                            brk.upperBound = breakValSS[count]
                            brk.label = breakLab[count]
                            count +=1

                    if descSplit == 'Turbidity':
                        print(f'Adjusting {descSplit}')
                        for brk in sym.colorizer.classBreaks:
                            brk.upperBound = breakValTurb[count]
                            brk.label = breakLab[count]
                            count+= 1

                    if descSplit == 'Chlorophyll':
                        print(f'Adjusting {descSplit}')
                        for brk in sym.colorizer.classBreaks:
                            brk.upperBound = breakValCHL[count]
                            brk.label = breakLab[count]
                            count+= 1

                    if descSplit == 'Altitude':
                        print(f'Adjusting {descSplit}')
                        for brk in sym.colorizer.classBreaks:
                            brk.upperBound = breakValAlt[count]
                            #brk.label = breakLab[count]

                            count+= 1

                    l.symbology = sym  
                    
            except Exception as e:
                print(f'Something went wrong with {l}')
    


            
    
                                                  
def export_raster():
    arcpy.management.CopyRaster("R_SN06255_20150210_Turbidity",
                                r"D:\NIWC_Eco\DELIVERABLES\SN06255_20150210\TIFF\Turbidity\SN06255_20150210_Turbidity.tif",
                                '',
                                0, 
                                "3.4e+38", 
                                "NONE", 
                                "ColormapToRGB",
                                "8_BIT_UNSIGNED", 
                                "NONE", 
                                "RGBToColormap",
                                "TIFF", 
                                "Transform",
                                "CURRENT_SLICE",
                                "NO_TRANSPOSE")
    
    
    
    
    arcpy.env.workspace = gdb_path
    for raster in raster_list:
        tif = raster[2:] + '.tif'
        mission_name = raster[2:18]
        variable = raster[19:]
        out_path = r"D:\NIWC_Eco\DELIVERABLES\{}\TIFF\{}\{}".format(mission_name, variable, tif)
        ope = os.path.exists(out_path)
        if ope == False:
            print(f"Exporting {no_prefix} Raster ")
            arcpy.management.CopyRaster(in_raster = raster,
                                        out_rasterdataset = out_tiff,
                                        onebit_to_eightbit = 'OneBitTo8Bit',
                                        colormap_to_RGB = None, 
                                        pixel_type = '32_BIT_SIGNED', 
                                        scale_pixel_value = None, 
                                        RGB_to_Colormap = None, 
                                        format = 'TIFF', 
                                        transform = None)
        else:
            print(f"{no_prefix} Tiff already exists")'''

hello world


In [7]:
#ctd_dir = os.listdir(ctd_directory_path)
#ctd_table_list = arcpy.ListTables('*_CTD*')
#ctd_extent_list = arcpy.ListFeatureClasses("*_ctdExtent", "Polygon")
#kearfott_dir = os.listdir(kearfott_directory_path)
#kearfott_table_list = arcpy.ListTables('*ke*')
#kearfott_extent_list = arcpy.ListFeatureClasses("*_altExtent", "Polygon")
#flntu_dir = os.listdir(flntu_directory_path)
#flntu_table_list = arcpy.ListTables('*_FLNTU*')
#flntu_extent_list = arcpy.ListFeatureClasses("*_flntuExtent", "Polygon")


'''##Class Breaks
classBreaks_Depth = np.linspace(10, 17, 10)
classBreaks_Depth = np.around(classBreaks_Depth, 1)
classBreaks_Depth = np.array(classBreaks_Depth)
classBreaks_Depth = list(classBreaks_Depth)

classBreaks_Temperature = np.linspace(12, 17, 10)
classBreaks_Temperature = np.around(classBreaks_Temperature, 1)
classBreaks_Temperature = np.array(classBreaks_Depth)
classBreaks_Temperature = list(classBreaks_Depth)

classBreaks_Salinity = np.linspace(32, 36, 10)
classBreaks_Salinity = np.around(classBreaks_Salinity, 3)
classBreaks_Salinity = np.array(classBreaks_Salinity)
classBreaks_Temperature = list(classBreaks_Salinity)

classBreaks_SoundSpeed = np.linspace(1493 ,1512, 10)
classBreaks_SoundSpeed = np.around(classBreaks_SoundSpeed, 3)
classBreaks_SoundSpeed = np.array(classBreaks_SoundSpeed)
classBreaks_Temperature = list(classBreaks_SoundSpeed)'''

ctd_directory_path = r"D:\BenWhitmore\EcoData\CTD"
flntu_directory_path = r"D:\BenWhitmore\EcoData\FLNTU"
kearfott_directory_path = r"D:\BenWhitmore\EcoData\Kearfott"

def add_kearfott():
    arcpy.env.workspace = gdb_path
    print('Adding Kearfott Tables')
    #print(kearfott_table_list)
    #default_gdb_path = tables_gdb_path
    for i in kearfott_dir:
        clean_i = i.split('.')[0]
        #print(clean_i)
        if clean_i in table_list:
            print(f"!!{clean_i} has already been added to map!!")
        else: 
            arcpy.env.workspace = kearfott_directory_path
            print(f"**Adding {clean_i}")
            arcpy.conversion.TableToGeodatabase(i, gdb_path)
def add_CTD():
    arcpy.env.workspace = gdb_path
    print('Adding CTD Tables')
    #print(kearfott_table_list)
    #default_gdb_path = tables_gdb_path
    for i in ctd_dir:
        clean_i = i.split('.')[0]
        #print(clean_i)
        if clean_i in table_list:
            print(f"!!{clean_i} has already been added to map!!")
        else: 
            arcpy.env.workspace = ctd_directory_path
            print(f"**Adding {clean_i}")
            arcpy.conversion.TableToGeodatabase(i, gdb_path)
def add_FLNTU():
    arcpy.env.workspace = gdb_path
    print('Adding FLNTU Tables')
    #print(kearfott_table_list)
    #default_gdb_path = tables_gdb_path
    for i in flntu_dir:
        clean_i = i.split('.')[0]
        #print(clean_i)
        if clean_i in table_list:
            print(f"!!{clean_i} has already been added to map!!")
        else: 
            arcpy.env.workspace = flntu_directory_path
            print(f"**Adding {clean_i}")
            arcpy.conversion.TableToGeodatabase(i, gdb_path)
def generate_kearfott_points():
    arcpy.env.workspace = gdb_path
    for i in kearfott_table_list:
        out_i = 'P_' +i[0:16] + '_K'
        #print(out_i)
        if out_i in kearfott_list:
            print(f"!!{out_i} Points Already Exists!!")
        else:
            print(f"**Creating {out_i} points")
            arcpy.defense.CoordinateTableToPoint(i, out_i, 'longitude', "DD_2", 'latitude', coordinate_system)

def generate_CTD_points():
    arcpy.env.workspace = gdb_path
    for i in kearfott_table_list:
        out_i = 'P_' +i[0:16] + '_C'
        #print(out_i)
        if out_i in ctd_list:
            print(f"!!{out_i} Points Already Exists!!")
        else:
            print(f"**Creating {out_i} points")
            arcpy.defense.CoordinateTableToPoint(i, out_i, 'longitude', "DD_2", 'latitude', coordinate_system)      
            
        
def generate_FLNTU_points():
    for i in flntu_table_list:
        out_i = i[0:16]+ '_flntu'
        print(f"Creating {i} points")
        arcpy.defense.CoordinateTableToPoint(i, out_i, 'longitude', "DD_2", 'latitude', coordinate_system)
            

def kear_extent():
    arcpy.env.workspace = kearfott_gdb_path
    for i in kearfott_list:
        mission_name = i[2:18] 
        out_i = 'E_' + mission_name + '_K'
        print(f"Creating Kearfott Extent")
        arcpy.management.MinimumBoundingGeometry(i, out_i, "CONVEX_HULL", "ALL", "", "MBG_FIELDS")
        
def CTD_extent():
    #arcpy.env.workspace = gdb_path
    for i in ctd_list:
        mission_name = i[0:16]
        out_i_extent = mission_name + '_ctdExtent'
        print("Creating Extent")
        arcpy.management.MinimumBoundingGeometry(i, out_i_extent, "CONVEX_HULL", "ALL", "", "MBG_FIELDS")
        
def FLNTU_extent():
    for i in flntu_list:
        mission_name = i[0:16] 
        out_i_extent = mission_name + '_flntuExtent'
        print(f"Creating FLNTU Extent")
        arcpy.management.MinimumBoundingGeometry(i, out_i_extent, "CONVEX_HULL", "ALL", "", "MBG_FIELDS")
        

def trim_kearfott():
    field = ["kearfott_alt"]
    print("Updating Kearfott Tables")
    for i in kearfott_list:
        with arcpy.da.UpdateCursor(i, field) as cursor:
            for row in cursor:
                if(row[0] < 0 or row[0] > 7):
                    cursor.deleteRow()
        
def trim_CTD():
    #arcpy.env.workspace = gdb_path
    fields = ["temperature", "salinity", "sound_speed", 'depth']
    for i in ctd_list:
        print('Updating Cursor')
        with arcpy.da.UpdateCursor(i, fields) as cursor:
            for row in cursor:
                if(row[0] < 0 or row[0] > 30):
                    cursor.deleteRow()
                if(row[1] < 30 or row[1] > 40):
                    cursor.deleteRow()
                if(row[2] < 0):
                    cursor.deleteRow()
                if(row[3] < 12):
                    cursor.deleteRow()
def trim_FLNTU():
    for i in flntu_list:
        field = ['depth']
        print('Updating Cursor')
        with arcpy.da.UpdateCursor(i, field) as cursor:
            for row in cursor:
                if(row[0] < 12 or row[0] > 25):
                    cursor.deleteRow()
                    
                    
def ctd():
    arcpy.env.workspace = gdb_path
    for i in ctd_list:
        for value in ctd_value_list:
            valueCap = value.capitalize()
            mission_name = i[0:16]
            idw = mission_name + '_IDW_' + valueCap
            con = mission_name +'_Contour_' + valueCap
            ras = mission_name +'_Raster_' + valueCap
            out_ras = os.path.join(gdb_path, ras)
            out_con = os.path.join(gdb_path, con)
            ##Root Directory Paths
            for extent in extent_list: 
                extent_name = mission_name+'_ctdExtent'
                extent_path = os.path.join(gdb_path, extent_name)
                if extent == extent_name: 
                    print(f"Creating {mission_name} {value} IDW ")
                    with arcpy.EnvManager(outputCoordinateSystem= coordinate_system, mask=extent):
                        arcpy.ga.IDW(i,value,idw,out_ras,cell_size,power,search_neighborhood,value)


def flntu():
    arcpy.env.workspace = gdb_path
    for i in flntu_list:
        for value in flntu_value_list:
            valueCap = value.capitalize()
            mission_name = i[0:16]
            idw = mission_name + '_IDW_' + valueCap
            con = mission_name +'_Contour_' + valueCap
            ras = mission_name +'_Raster_' + valueCap
            out_ras = os.path.join(gdb_path, ras)
            for extent in extent_list: 
                extent_name = mission_name+'_flntuExtent'
                extent_path = os.path.join(gdb_path, extent_name)
                if extent == extent_name: 
                    print(f"Creating {mission_name} {value} IDW ")
                    with arcpy.EnvManager(outputCoordinateSystem= coordinate_system, mask=extent):
                        arcpy.ga.IDW(i,value,idw,out_ras,cell_size,power,search_neighborhood,value)

                        

def kearfott():
    arcpy.env.workspace = gdb_path
    for i in kearfott_list:
        for value in kearfott_value_list:
            valueCap = value.capitalize()
            mission_name = i[0:16]
            idw = mission_name + '_IDW_' + valueCap
            ras = mission_name +'_Raster_' + valueCap
            out_ras = os.path.join(gdb_path, ras)
            for extent in extent_list: 
                extent_name = mission_name+'_kearfottExtent'
                extent_path = os.path.join(gdb_path, extent_name)
                if extent == extent_name: 
                    print(f"Creating {mission_name} {value} IDW ")
                    with arcpy.EnvManager(outputCoordinateSystem= coordinate_system, mask=extent):
                        arcpy.ga.IDW(i,value,idw,out_ras,cell_size,power,search_neighborhood,value)
            
            

In [12]:
classBreaks_Depth = np.linspace(10, 17, 5)
classBreaks_Depth = np.around(classBreaks_Depth, 1)
classBreaks_Depth = np.array(classBreaks_Depth)
classBreaks_Depth = list(classBreaks_Depth)
print(classBreaks_Depth)
classBreaks_Temperature = np.linspace(12, 17, 5)
classBreaks_Temperature = np.around(classBreaks_Temperature, 1)
classBreaks_Temperature = np.array(classBreaks_Temperature)
classBreaks_Temperature = list(classBreaks_Temperature)
print(classBreaks_Temperature)
classBreaks_Salinity = np.linspace(33.2, 33.4, 5)
classBreaks_Salinity = np.around(classBreaks_Salinity, 5)
classBreaks_Salinity = np.array(classBreaks_Salinity)
classBreaks_Salinity = list(classBreaks_Salinity)
print(classBreaks_Salinity)
classBreaks_SoundSpeed = np.linspace(1493 ,1512, 5)
classBreaks_SoundSpeed = np.around(classBreaks_SoundSpeed, 3)
classBreaks_SoundSpeed = np.array(classBreaks_SoundSpeed)
classBreaks_SoundSpeed = list(classBreaks_SoundSpeed)
print(classBreaks_SoundSpeed)
classBreaks_Turbidity = np.linspace(100 ,200, 5)
classBreaks_Turbidity = np.around(classBreaks_Turbidity, 0)
classBreaks_Turbidity = np.array(classBreaks_Turbidity)
classBreaks_Turbidity = list(classBreaks_Turbidity)
print(classBreaks_Turbidity)
classBreaks_Chlorophyll = np.linspace(0.4 ,.8, 5)
classBreaks_Chlorophyll = np.around(classBreaks_Chlorophyll,4)
classBreaks_Chlorophyll = np.array(classBreaks_Chlorophyll)
classBreaks_Chlorophyll = list(classBreaks_Chlorophyll)
print(classBreaks_Chlorophyll)

[10.0, 11.8, 13.5, 15.2, 17.0]
[12.0, 13.2, 14.5, 15.8, 17.0]
[33.2, 33.25, 33.3, 33.35, 33.4]
[1493.0, 1497.75, 1502.5, 1507.25, 1512.0]
[100.0, 125.0, 150.0, 175.0, 200.0]
[0.4, 0.5, 0.6, 0.7, 0.8]
